In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import Markdown, HTML, display
import json
import yaml

import libconfig as cfg

In [3]:
def describe_function( function ):
    display(Markdown("**{}( ):**".format(function.__name__)))
    display(HTML(function.__doc__.strip("\n").replace("\n", "<br/>").replace(" ", "&nbsp;").replace(";:", ";<u>:").replace(":&", ":</u>&")))

# Registering the options in your new library

Let's assume that one decides to declare the options in `core/core_init.py` inside the library.  
Obviously, the file must be the first to be imported in the main `__init__.py` of the library if it is meant to be accessed by any further part of it.  
The core configuration file might look like this:

In [4]:
import os

from libconfig import *

# Reset before starting is good, it avoids issues when working with jupyter and %autoreload.
# Otherwise an error will be raised trying to register again already registered options.
reset_options()

# Register the options for the library, here some examples of the different types.
register_option("number", "integer", 5, "int", "this is an integer example", True)
register_option("number", "float", 3.4, "float", "this is a float example", False)
register_option("no_number", "boolean", True, "bool", "this is a boolean example", False)
register_option("no_number", "text", "the_string", "string", "this is a string example", False)
register_option("paths", "in", "demo.ipynb", "path_in", "this is a path example that has to exist", False)
register_option("paths", "out.json", "configs.json", "path_out",
                    "this is a path example that does not have to exist", False)
register_option("paths", "out.yaml", "configs.yaml", "path_out",
                    "this is a path example that does not have to exist", False)

# Let's assume one wants to give the option to generate a user's configuration file.
# First time the library is called, it can create this config file with the current defaults.
# The user can change those for further runs. Ideally, the file would be something such as:
config_file = os.path.join(os.getenv("HOME"), ".libraryname")
# But, to avoid raising an IOError, for this example we point to a file in the demo
config_file = "configs.yaml"

# Either make or read from the file.
if not os.path.isfile(config_file):
    write_options_to_YAML( config_file )
else:
    set_options_from_YAML( config_file )

# Finally, register_option and reset_option are taken out from the global view so that they are not
# imported with the rest of the functions. This way the user can not access to them when importing
# the library and has to work through the rest of the available functions.
for name in ["register_option", "reset_options"]:
    del globals()[name]

**Be aware that locked options cannot be changed! Not even through the global configuration file**

The main functions called here are:

In [5]:
describe_function( cfg.register_option )
describe_function( cfg.reset_options )

**register_option( ):**

**reset_options( ):**

and one can see the current options with: (see how `number_float` has a different value than the default, loaded through the config file)

In [6]:
show_options()

,primary-key,secondary-key,value,type,default,locked,description
0,number,integer,5,int,5,True,this is an integer example
1,number,float,12,float,3.4,False,this is a float example
2,no_number,boolean,1,bool,1,False,this is a boolean example
3,no_number,text,the_string,string,the_string,False,this is a string example
4,paths,in,demo.ipynb,path_in,demo.ipynb,False,this is a path example that has to exist
5,paths,out.json,configs.json,path_out,configs.json,False,this is a path example that does not have to exist
6,paths,out.yaml,configs.yaml,path_out,configs.yaml,False,this is a path example that does not have to exist


alternatively to **YAML**, configuration files can also be defined with **JSON**

In [7]:
describe_function( write_options_to_YAML )
describe_function( write_options_to_JSON )
describe_function( set_options_from_YAML )
describe_function( set_options_from_JSON )

**write_options_to_YAML( ):**

**write_options_to_JSON( ):**

**set_options_from_YAML( ):**

**set_options_from_JSON( ):**

Functionally, options can be accessed with `get_option` and changed with `set_option`

In [8]:
describe_function( get_option )
describe_function( set_option )

**get_option( ):**

**set_option( ):**

Trying to set a value that is not the proper one in an option will result in an error.  
So will trying to assign a value to a non-registered option or to change a *locked* option.

In [9]:
try:
    set_option( "number", "integer", 5.5 )
except ValueError, e:
    print e

Value must have type '<type 'int'>'


In [10]:
try:
    set_option( "number", "random", 5.5 )
except KeyError, e:
    print e

'number.random option is not registered'


In [11]:
try:
    set_option( "number", "integer", 5.5 )
except ValueError, e:
    print e

Value must have type '<type 'int'>'


All available functions are listed here; they are better explained in the [API]()

In [13]:
[x for x in dir(cfg) if not x.startswith("_") and "_" in x]

['get_option',
 'get_option_default',
 'get_option_description',
 'lock_option',
 'register_option',
 'reset_option',
 'reset_options',
 'set_option',
 'set_options_from_JSON',
 'set_options_from_YAML',
 'set_options_from_dict',
 'show_options',
 'write_options_to_JSON',
 'write_options_to_YAML']